Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

In [20]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [21]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


In [24]:
def normalize_database(non_normalized_db_filename):
     
    #degrees table
    degreeval = df.Degree.unique()
    degreevalues = []
    for i in degreeval:
        degreevalues.append(i)    
    
    
    #students table
    names = [i.strip().split(",") for i in df.Name]
    first_name = []
    last_name = []
    for i in names:
        last_name.append(i[0])
        first_name.append(i[1].strip())
    
    degrees = [i for i in df.Degree]
    studentID = [i for i in df.StudentID]

    zippedstudentstable = [(i,j,k,l) for i, j, k, l in zip(studentID,first_name,last_name,degrees)]
    
    #exams table
    listy = []
    for i in df.Exams:
        listy.append(i)

    list1 = []    
    for i in listy:
        list1.append(i.split(','))
    
    list2 = [] 
    for i in list1:
        for z in i:
            list2.append(z.strip())
            
    examlist = []
    for i in list2:
        if i not in examlist:
            examlist.append(i.strip())
        else:
            continue
        
    examlist = [i.split() for i in examlist]
    examss = []
    year = []
    
    for i in examlist:
        examss.append(i[0].strip())
        year.append(i[1].strip())
    
    year = [(i[1:-1]) for i in year]

    zippedexamyear = [(i.strip(),int(j)) for i,j in zip(examss,year)]
    
    #studentexamscores table

    asd = []
    for i in df.Exams:
        asd.append(i.split(","))

    examsstudent =  []
    
    for i in asd:
        for z in i:
            examsstudent.append(z[0:-6].strip())
        
    #print(examsstudent)    
        
    scored = []
    for i in df.Scores:
        scored.append(i.split(","))
    
    scored1 = []
    for z in scored:
        for a in z:
            scored1.append(a.strip())
        
    number = []
    for z in df.Scores:
        number.append((len(z.strip().split(","))))

    studentidnumbers = []
    for i,j in zip(studentID, number):
        for z in ([i]*j):
            studentidnumbers.append(z)
        
        
        
    studentexamscores = [(p,i,j,k) for p,i,j,k in zip(range(1,585),studentidnumbers,examsstudent,scored1)]

    
    conn = create_connection('normalized.db',delete_db=True)


    with conn:
        create_exam_table_sql = """ CREATE TABLE [Exams] (  
        [Exam] VARCHAR  NOT NULL,
        [Year] INT NOT NULL,
        PRIMARY KEY (Exam)
        );
        """
        create_Degrees_sql = """ CREATE TABLE [Degrees] (  
        [Degree] VARCHAR  NOT NULL,
        PRIMARY KEY (Degree)
        );
        """
        create_students_sql = """ CREATE TABLE [Studentss] (  
        [StudentID] INT  NOT NULL,
        [First_Name] VARCHAR NOT NULL,
        [Last_Name] VARCHAR NOT NULL,
        [Degree] VARCHAR NOT NULL,
        PRIMARY KEY (StudentID),
        FOREIGN KEY(Degree) references Degrees(Degree)
    
        );
        """
    
        create_studentexam_scores_sql = """ CREATE TABLE [Student_examscores] (  
        [PK] VARCHAR  NOT NULL,
        [StudentID] INT NOT NULL,
        [Exam] VARCHAR NOT NULL,
        [Score] INT NOT NULL,
        PRIMARY KEY (PK),
        FOREIGN KEY(StudentID) references Studentss(StudentID),
        FOREIGN KEY(Exam) references Exams(Exam)
        );
        """
    
    
        def insert_Exams(conn, values):
            sql = """ INSERT INTO Exams VALUES(?, ?) """
            cur = conn.cursor()
            cur.execute(sql, values)
            return cur.lastrowid


        def insert_Degrees(conn, values):
            sql = """ INSERT INTO Degrees VALUES(?) """
            cur = conn.cursor()
            cur.execute(sql, values)
            return cur.lastrowid

        def insert_students(conn, values):
            sql = """ INSERT INTO Studentss VALUES(?, ?, ?, ?) """
            cur = conn.cursor()
            cur.execute(sql, values)
            return cur.lastrowid

        def insert_Student_examscores(conn, values):
            sql = """ INSERT INTO Student_examscores VALUES(?, ?, ?, ?) """
            cur = conn.cursor()
            cur.execute(sql, values)
            return cur.lastrowid
        
        
        create_table(conn, create_Degrees_sql)
        create_table(conn, create_exam_table_sql)
        create_table(conn, create_students_sql)
        create_table(conn, create_studentexam_scores_sql)
        
        for i in degreevalues:
            insert_Degrees(conn, (i,))
    
        for i in zippedstudentstable:
            insert_students(conn,i)
        
        for i in zippedexamyear:
            insert_Exams(conn,i)
        
        for i in studentexamscores:
            insert_Student_examscores(conn,i)
   
    
    
    #     Normalize 'non_normalized.db'
#     Call the normalized database 'normalized.db'
#     Function Output: No outputs
#     Requirements:
#     Create four tables
#     Degrees table has one column:
#         [Degree] column is the primary key
    
#     Exams table has two columns:
#         [Exam] column is the primary key column
#         [Year] column stores the exam year
    
#     Students table has four columns:
#         [StudentID] primary key column 
#         [First_Name] stores first name
#         [Last_Name] stores last name
#         [Degree] foreign key to Degrees table
        
#     StudentExamScores table has four columns:
#         [PK] primary key column,
#         [StudentID] foreign key to Students table,
#         [Exam] foreign key to Exams table ,
#         [Score] exam score


    # YOUR CODE HERE
    #raise NotImplementedError()
        
    
    

In [25]:
normalize_database('non_normalized.db')
conn = create_connection('normalized.db')


In [26]:
def ex1():
    # Write an SQL statement that SELECTs all rows from the `Exams` table and sort the exams by Year
    # output columns: exam, year
    sql_statement = 'select exam,year from exams order by year,exam'
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [7]:
sql_statement = ex1()
data = pd.read_csv("ex1.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year
0,exam1,2016
1,exam2,2017
2,exam6,2017
3,exam7,2017
4,exam3,2018
5,exam8,2018
6,exam9,2018
7,exam10,2019
8,exam4,2019
9,exam5,2020


In [8]:
def ex2():
    # Write an SQL statement that SELECTs all rows from the `Degrees` table and sort the degrees by name
    # output columns: degree
    sql_statement = 'select * from degrees order by degree'
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [9]:
sql_statement = ex2()
data = pd.read_csv("ex2.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree
0,graduate
1,undergraduate


In [10]:
def ex3():
    # Write an SQL statement that counts the numbers of gradate and undergraduate students
    # output columns: degree, count_degree
    sql_statement = 'select degree, count(degree) as count_degree  from studentss group by degree order by degree'
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [11]:
sql_statement = ex3()
data = pd.read_csv("ex3.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,count_degree
0,graduate,35
1,undergraduate,65


In [12]:
def ex4():
    # Write an SQL statement that calculates the exam averages for exams; sort by average in descending order.
    # output columns: exam, year, average
    # round to two decimal places
    
    sql_statement = 'select a.exam, year, round(avg(score),2) as average from Student_examscores a inner join exams b on a.exam = b.exam group by a.exam order by average desc'
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [13]:
sql_statement = ex4()
data = pd.read_csv("ex4.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year,average
0,exam3,2018,86.16
1,exam8,2018,80.18
2,exam1,2016,75.09
3,exam7,2017,68.96
4,exam2,2017,64.28
5,exam6,2017,61.80
6,exam4,2019,54.04
7,exam5,2020,45.56
8,exam10,2019,44.68
9,exam9,2018,39.43


In [14]:
def ex5():
    # Write an SQL statement that calculates the exam averages for degrees; sort by average in descending order.
    # output columns: degree, average 
    # round to two decimal places
    
    sql_statement = 'select degree, round(avg(score),2) as average from Student_examscores a inner join studentss b on a.studentid = b.studentid group by degree'
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [15]:
sql_statement = ex5()
data = pd.read_csv("ex5.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,average
0,graduate,62.50
1,undergraduate,61.63


In [16]:
def ex6():
    # Write an SQL statement that calculates the exam averages for students; sort by average in descending order. Show only top 10 students
    # output columns: first_name, last_name, degree, average
    # round to two decimal places
    sql_statement = 'select first_name, last_name, degree, round(avg(score),2) as average from studentss a inner join Student_examscores b on a.studentid = b.studentid group by a.studentid order by avg(score) desc limit 10'
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [17]:
sql_statement = ex6()
data = pd.read_csv("ex6.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,First_Name,Last_Name,Degree,average
0,John,Washington,graduate,83.00
1,Robert,Andrade,undergraduate,82.00
2,Paul,Smith,undergraduate,79.00
3,Jason,Delgado,undergraduate,77.50
4,Jason,Thompson,graduate,75.00
5,Calvin,Perez,undergraduate,75.00
6,Calvin,Martinez,undergraduate,74.50
7,Billy,Palmer,undergraduate,74.38
8,Matthew,King,undergraduate,73.56
9,Debra,Pratt,undergraduate,72.60
